## CHEME 5660: Building a Trading Bot using Model-Free Reinforcement Learning

### Introduction
In this example we'll use [Q-learning](https://en.wikipedia.org/wiki/Q-learning), a model-free reinforcement learning approach, to In this example, we'll use [Q-learning](https://en.wikipedia.org/wiki/Q-learning), a model-free reinforcement learning approach, to build a trading bot for stocks in the `CHEME-5660 portfolio`. In particular, we've downloaded `5 min` Open High Low Close (OHLC) data for the 150 tickers in the `CHEME 5660 portfolio` from [Polygon.io](https://polygon.io). From this data, we'll use the Q-learning approach from [Chapter 17 of Kochenderfer et al. (2022)](https://algorithmsbook.com) to estimate the $Q(s, a)$ table; once we have the $Q(s, a)$ table, we can estimate the policy $\pi(s)$:

$$\pi(s) = \text{arg}\max_{a}Q(s,a)$$

We estimate the $Q(s,a)$ table through experimentation, where we update $Q(s,a)$ after each experiment using the update rule:

$$Q(s,a)\leftarrow{Q(s,a)}+\alpha\left(r+\gamma\max_{a^{\prime}}Q(s^{\prime},a^{\prime}) - Q(s,a)\right)$$

Background reading/viewing on Reinforcement Learning and Model-free Reinforcement Learning:
* [Chapter 17: Mykel J. Kochenderfer, Tim A. Wheeler, Kyle H. Wray "Algorithms for Decision Making", MIT Press 2022](https://algorithmsbook.com)
* [Stanford CS234: Reinforcement Learning (2019), Lecture 4](https://www.youtube.com/playlist?list=PLoROMvodv4rOSOPzutgyCTapiGlY2Nd8u)
* [Stanford CS221: Artificial Intelligence: Principles and Techniques (2019), Lecture 8](https://www.youtube.com/playlist?list=PLoROMvodv4rO1NB9TD4iUZ3qghGEGtqNX) 

### Example setup

In [1]:
import Pkg; Pkg.activate("."); Pkg.resolve(); Pkg.instantiate();

  Activating project at `~/Desktop/julia_work/CHEME-5660-Markets-Mayhem-Example-Notebooks/jupyter-notebooks/CHEME-5660-Q-Learning-TradeBot-notebook`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Markets-Mayhem-Example-Notebooks/jupyter-notebooks/CHEME-5660-Q-Learning-TradeBot-notebook/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Markets-Mayhem-Example-Notebooks/jupyter-notebooks/CHEME-5660-Q-Learning-TradeBot-notebook/Manifest.toml`


In [2]:
# load req packages -
using DataFrames
using Dates
using FileIO
using JLD2
using PrettyTables
using Distributions
using Statistics
using DataFrames
using Plots
using Colors
using MLJLinearModels

# setup paths -
const _ROOT = pwd();
const _PATH_TO_DATA = joinpath(_ROOT, "data");

#### Load the example code library
The call to the `include` function loads the `CHEME-5660-Example-CodeLib.jl` library into the notebook; this library contains types and functions we use during the example. In particular, we encode the online $Q(s,a)$ update routine listed as `Algorithm 17.2` from [Kochenderfer et al (2022)](https://algorithmsbook.com).

In [3]:
include("CHEME-5660-Example-CodeLib.jl");

### Setup constants and other resources

#### Load and partition the OHLC price data set
This data is `5 min` data, meaning we have Open High Low Close (OHLC) data for the 150 tickers in the `CHEME 5660 portfolio` every five minutes for one week of trading days. We'll use this data to estimate the trade policy of our agent. 

In [4]:
# what ticker do we want to explore?
ticker_symbol = "AMD";

# load the JLD2 portfolio data file -
price_data_dictionary = load(joinpath(_PATH_TO_DATA, "CHEME-5660-Portfolio-Q-learning-5min-11-20-22.jld2"))["dd"];

# we have these ticker symbols in our data set -
ticker_symbol_array = sort(keys(price_data_dictionary) |> collect);

# Partition the data into a training and prediction set
(price_training_dict, price_prediction_dict) = partition(price_data_dictionary; fraction=0.90);

# this version uses all the data for training -
# df_training = price_data_dictionary[ticker_symbol];

# this version uses only some of the data for training 
df_training = price_training_dict[ticker_symbol];
df_prediction = price_prediction_dict[ticker_symbol];

##### Constants

In [5]:
# how many days of historical data are we using?
d = 1;       # we nₐ buy shares of XYZ
nₐ = 1.0;    # how many shares do we want to buy, sell each day
δ = 0.50;    # z-score cutoff

# setup actions states -
actions = [1,2,3]  ; # buy, sell, hold
states = [1,2,3,4] ; # states defined below -

# parameters for the Q(s,a) estimation -
ϵ = 0.45;
number_of_trials = 500;

#### Define discrete state classes
Let the state of our decision process be the share price of the underlying asset `XYZ`. However, the share price is (arguably) a continuous state variable, and we need to have discrete states. Toward this challenge, let's train a [Multiclass classifier](https://en.wikipedia.org/wiki/Multiclass_classification) that takes price as an input and gives back an integer class $\mathcal{C} = \left\{1,2,\dots,c\right\}$; where we build the following classes:

* The share price $S_{t}$ belongs to `class 1` if it is close to an expected price $\mathbb{E}\left(S\right)$, but larger than the expected price $S_{t}>\mathbb{E}\left(S\right)$. 
* The share price $S_{t}$ belongs to `class 2` if it is much larger than an expected price $\mathbb{E}\left(S\right)$, i.e., $S_{t}\gg\mathbb{E}\left(S\right)$. 
* The share price $S_{t}$ belongs to `class 3` if it is close to an expected price $\mathbb{E}\left(S\right)$, but smaller than the expected price $S_{t}<\mathbb{E}\left(S\right)$.
* The share price $S_{t}$ belongs to `class 4` if it is much smaller than an expected price $\mathbb{E}\left(S\right)$, i.e., $S_{t}\ll\mathbb{E}\left(S\right)$.

The notion of `close`, `smaller` or `larger` needs to be more concrete; let's base these distances on the [z-score cutoff](https://en.wikipedia.org/wiki/Standard_score) parameter $\delta$ defined in the constants section.

In [6]:
# fit a distribution to vwap data -
normal_price_distribution = fit_mle(Normal, df_training[:,:volume_weighted_average_price]);

# get parameters -
θ = params(normal_price_distribution);

# setup price -
Sₒ = θ[1];
σ̂ = θ[2];

# print -
println("Long-term price of $(ticker_symbol) is Sₒ = $(Sₒ) USD/share with σ̂ = $(σ̂) USD/share")

Long-term price of AMD is Sₒ = 74.47168145933033 USD/share with σ̂ = 1.4059735498911663 USD/share


In [7]:
# build a labled training data set:
  
# initialize -
number_of_training_examples = nrow(df_training);
number_of_column_labels = 2
labeled_training_data = Array{Float64,2}(undef, number_of_training_examples,1);
label_array = Array{Int64,1}(undef, number_of_training_examples);

for i ∈ 1:number_of_training_examples
    
    # get the vwap -
    vwap_value = df_training[i,:volume_weighted_average_price]; #vwap = volume weighted average price
    labeled_training_data[i,1] = vwap_value;
    label_array[i] = state(vwap_value; μ = Sₒ, σ = σ̂, δ = δ)
end

# build a multiclass classifier -
mc_classifier_model = MultinomialRegression();
theta = MLJLinearModels.fit(mc_classifier_model, labeled_training_data, label_array);
W = reshape(theta,2,length(states));

##### In-sample validation of the state classifier

In [8]:
# compute the percent correct classification in sample -
tmp_classification_array = Array{Int64,1}()
for i ∈ 1:number_of_training_examples
    
    predicted_state_class = state(labeled_training_data[i,1], W);
    actual_state_class = label_array[i];
    
    if (predicted_state_class == actual_state_class)
        push!(tmp_classification_array,1);
    else
        push!(tmp_classification_array,0);
    end
end

# how many 1's -
correct_classification_fraction = (1/number_of_training_examples)*sum(tmp_classification_array);

# correct -
println("In sample correct classification fraction: $(correct_classification_fraction)")

In sample correct classification fraction: 1.0


##### Out-of-sample validation of the state classifier

In [9]:
# compute the percent correct classification in sample -
tmp_ous_classification_array = Array{Int64,1}()
number_of_prediction_examples = nrow(df_prediction);
for i ∈ 1:number_of_prediction_examples
    
    # grab the price -
    vwap_value = df_prediction[i,:volume_weighted_average_price];
    
    # compute the predicted class using the classifier -
    predicted_state_class = state(vwap_value, W);
    
    # compute the actual class using the manual method -
    actual_state_class = state(vwap_value; μ = Sₒ, σ = σ̂, δ = δ);
    
    if (predicted_state_class == actual_state_class)
        push!(tmp_ous_classification_array,1);
    else
        push!(tmp_ous_classification_array,0);
    end
end

# how many 1's -
correct_classification_ous_fraction = (1/number_of_prediction_examples)*sum(tmp_ous_classification_array);

# output -
println("Out of sample correct classification fraction: $(correct_classification_ous_fraction)")

Out of sample correct classification fraction: 1.0


#### Can we break the classifier?

In [10]:
# pick an index at random -
idx_quick_look = 41;

# grab the price -
vwap_value = df_prediction[idx_quick_look,:volume_weighted_average_price];
# vwap_value = df_training[idx_quick_look,:volume_weighted_average_price]; # look at training ...

# compute the predicted class using the classifier -
predicted_state_class = state(vwap_value, W);
    
# compute the actual class using the manual method -
actual_state_class = state(vwap_value; μ = Sₒ, σ = σ̂, δ = δ);

# println -
println("vwap = $(vwap_value) USD/share is class: $(actual_state_class) and predicted to be class: $(predicted_state_class)")

vwap = 73.5891 USD/share is class: 4 and predicted to be class: 4


### Online estimation of the Q-table using Q-learning 
Now that we have our state classes (and our state classifier) let's turn our attention to the online estimation of the state-action-value matrix, otherwise known as the $ Q$ function. Imagine a scenario in which we don't know the transition matrix $T_{a}(s,s^{\prime})$ or the rewards array $R(s, a)$. Instead, we let our system learn by trying different actions in different states and seeing what reward we get. Thus, we allow our agent to learn by example. 

In this example, we allow the agent to execute random actions (exploration) or execute its best guess of the optimal move, and then we observe the reward received. 
* The action space in this example is $\mathcal{A}=\left\{\mathtt{buy}, \mathtt{sell}, \mathtt{hold}\right\}$; thus, we either buy $n_{a}$ shares of `XYZ`, sell $n_{a}$ shares of `XYZ` or do nothing. 
* The state space $\mathcal{S}$ is described above: $s_{1}$: current share close to but above some long-term expected price, $s_{2}$: current share price well above some long-term expected price, $s_{3}$: current share close to but below some long-term expected price and $s_{4}$: current share price well below some long-term expected price.
* The reward we have chosen is the liquidation value on a per share basis of our position, i.e., what value could we get if we sold everything at the _next available market price_. In our case, we are using `5 min` data.

#### Computational details
* We do a random action $\epsilon$ of the time; otherwise, we let the agent execute its best estimate of the optimal action.
* We run `number_of_trials`, collect the $Q$-function for each trial and estimate a policy $\pi(s)$ from the $Q(s, a)$ table for each trial. 
* After finishing `number_of_trials` and analyzing the data, we build a table with the recommended action for each state.
* The update function was taken from `Algorithm 17.2` from [Kochenderfer et al. (2022)](https://algorithmsbook.com)
* The current implementation of the code is slow. You've been warned. 

#### Interesting question:
* I can imagine several `expert` policies, e.g., `buy the dip, sell the bounce`, a policy of 2,2,1,1. Alternatively, you could get `don't play small ball`, meaning wait for large swings to do anything (this has a policy of 3,2,3,1). Do we recover either of these policies? 

In [11]:
# setup ϵ sim -
policy_array = Array{Int64,2}(undef, 4, number_of_trials);

# setup categorical distribution for drawing a random action -
action_distribution = Categorical([0.36,0.32,0.32]);

# initialize an empty ledger -
ledger_df = DataFrame(
    time = DateTime[],
    n = Float64[],
    price = Float64[],
    s = Int64[],
    action = Int64[]
);

for t ∈ 1:number_of_trials
    
    # clear the data frame -
    empty!(ledger_df);
    
    # initialize an empty Q -
    Q_array = Array{Float64,2}(undef, length(states), length(actions));
    fill!(Q_array, 0.0);
    
    # initial policy -> bias toward expert policy
    Q_array[1,2] = 10.0;
    Q_array[2,2] = 10.0;
    Q_array[3,1] = 10.0;
    Q_array[4,1] = 10.0;
    
    # build Q model -
    QMODEL = QLearningModel();
    QMODEL.γ = 0.75;
    QMODEL.α = 0.10;
    QMODEL.𝒮 = states;
    QMODEL.𝒜 = actions;
    QMODEL.Q = Q_array;
    
    # we buy 100 shares at initial price in the data -
    transaction = (
        time = df_training[1,:timestamp],
        n = 100.0,
        action = 1,
        price = price(df_training,1),
        s = state(price(df_training,1),W) 
    );
    push!(ledger_df, transaction)
    
    # main random simulation -
    for i ∈ 2:(nrow(df_training) - 1)

        # get data from the df_training -
        p = price(df_training, i);
        p′ = price(df_training, i+1);

        # convert that to the current state -
        s = state(p, W);
        s′ = state(p′, W);

        # roll a random number -
        aᵢ = 3; # default is to hold
        if (rand() <= ϵ)
            # roll a random action - 
            aᵢ = rand(action_distribution);
        else
            # ok, what action does my best guess say that I should take?
            policy = π(QMODEL.Q);
            aᵢ = policy[s];
        end
        
        # ok, if we have a sell, then we need to check to make sure we have the shares?
        total_number_of_shares_check = compute_position_size(ledger_df);
        if (aᵢ == 2 && total_number_of_shares_check < 10.0)
            
            # override - buy 1 shares -
            transaction = (
                time = df_training[i,:timestamp],
                n = 2,
                action = 1,
                price = p, 
                s = s
            );
            push!(ledger_df, transaction)
            
        else
            
            # build a transaction object -
            transaction = (
                time = df_training[i,:timestamp],
                n = nₐ,
                action = aᵢ,
                price = p, 
                s = s
            );
            push!(ledger_df, transaction)
        end
        
        # compute the total number of shares -
        total_number_of_shares = compute_position_size(ledger_df);
        
        if (total_number_of_shares < 0)
            @show total_number_of_shares
        end
        
        # we've update the ledger - compute the return per share if we sold everything at the price for the next time step -
        Rᵢ = -1*max(0,total_number_of_shares)*liquidate(ledger_df, p′);

        # update the QMODEL -
        update!(QMODEL, s, aᵢ, Rᵢ, s′);
    end
    
    pvec = π(Q_array)
    policy_array[1,t] = pvec[1]
    policy_array[2,t] = pvec[2]
    policy_array[3,t] = pvec[3]
    policy_array[4,t] = pvec[4]
end

In [12]:
# show the policy table -

# initialize -
policy_table_data_array = Array{Any,2}(undef, 4, 6);
for s ∈ 1:length(states)
    
    # compute -
    policy_table_data_array[s,1] = s;
    
    # compute the fraction of a₁, a₂ and a₃ -
    idx_a₁ = findall(x->x==1, policy_array[s,:]);
    idx_a₂ = findall(x->x==2, policy_array[s,:]);
    idx_a₃ = findall(x->x==3, policy_array[s,:]);
    
    # compute the fraction -
    policy_table_data_array[s,4] = (length(idx_a₁)/number_of_trials);
    policy_table_data_array[s,5] = (length(idx_a₂)/number_of_trials);
    policy_table_data_array[s,6] = (length(idx_a₃)/number_of_trials);
end

# put expert in -
policy_table_data_array[1,2] = 2;
policy_table_data_array[2,2] = 2;
policy_table_data_array[3,2] = 1;
policy_table_data_array[4,2] = 1;

# put recomended in -
policy_table_data_array[1,3] = argmax(softmax(Vector{Float64}(policy_table_data_array[1,4:end])));
policy_table_data_array[2,3] = argmax(softmax(Vector{Float64}(policy_table_data_array[2,4:end])));
policy_table_data_array[3,3] = argmax(softmax(Vector{Float64}(policy_table_data_array[3,4:end])));
policy_table_data_array[4,3] = argmax(softmax(Vector{Float64}(policy_table_data_array[4,4:end])));

# header -
header_data_policy_table = (
    ["Data (N=$(number_of_trials)): $(ticker_symbol) ", "", "", "", "", ""],
    ["s", "π(s)", "π̂(s)","a₁ (buy)", "a₂ (sell)", "a₃ (hold)"]
);

# display -
pretty_table(policy_table_data_array; header=header_data_policy_table)

┌────────────────────┬──────┬──────┬──────────┬───────────┬───────────┐
│ Data (N=500): AMD  │      │      │          │           │           │
│                  s │ π(s) │ π̂(s) │ a₁ (buy) │ a₂ (sell) │ a₃ (hold) │
├────────────────────┼──────┼──────┼──────────┼───────────┼───────────┤
│                  1 │    2 │    2 │    0.308 │      0.39 │     0.302 │
│                  2 │    2 │    2 │    0.202 │      0.49 │     0.308 │
│                  3 │    1 │    2 │    0.336 │      0.34 │     0.324 │
│                  4 │    1 │    1 │    0.496 │     0.212 │     0.292 │
└────────────────────┴──────┴──────┴──────────┴───────────┴───────────┘


__Table__: Recommended policy and the fraction of trials resulting in aᵢ for state s. 

In [13]:
# build a local policy array -
local_policy_array = Array{Int64,1}(undef, length(states));
for s ∈ 1:length(states)
    local_policy_array[s] = policy_table_data_array[s,3]; # recommended policy is col 3
end

# save policy object so we can test later -
policy_model = build(TradeBotModel; policy = local_policy_array, ticker=ticker_symbol, W = W);

# dump policy_model to disk -
filename = "Policy-$(ticker_symbol)-Model-10.jld2";
save(joinpath("data",filename), Dict("model"=>policy_model))

In [14]:
# check:
for i ∈ 1:(nrow(df_training) - 1)
    
    total_number_of_shares = compute_position_size(ledger_df[1:i,:]);
    @show (i, total_number_of_shares)
    
end

(i, total_number_of_shares) = (1, 100)
(i, total_number_of_shares) = (2, 99)
(i, total_number_of_shares) = (3, 99)
(i, total_number_of_shares) = (4, 100)
(i, total_number_of_shares) = (5, 101)
(i, total_number_of_shares) = (6, 102)
(i, total_number_of_shares) = (7, 103)
(i, total_number_of_shares) = (8, 102)
(i, total_number_of_shares) = (9, 103)
(i, total_number_of_shares) = (10, 102)
(i, total_number_of_shares) = (11, 101)
(i, total_number_of_shares) = (12, 101)
(i, total_number_of_shares) = (13, 102)
(i, total_number_of_shares) = (14, 102)
(i, total_number_of_shares) = (15, 102)
(i, total_number_of_shares) = (16, 101)
(i, total_number_of_shares) = (17, 101)
(i, total_number_of_shares) = (18, 100)
(i, total_number_of_shares) = (19, 101)
(i, total_number_of_shares) = (20, 100)
(i, total_number_of_shares) = (21, 100)
(i, total_number_of_shares) = (22, 101)
(i, total_number_of_shares) = (23, 100)
(i, total_number_of_shares) = (24, 101)
(i, total_number_of_shares) = (25, 101)
(i, total_n

### Additional Resources
* [Chapter 17: Mykel J. Kochenderfer, Tim A. Wheeler, Kyle H. Wray "Algorithms for Decision Making", MIT Press 2022](https://algorithmsbook.com)

### Disclaimer and Risks
__This content is offered solely for training and  informational purposes__. No offer or solicitation to buy or sell securities or derivative products, or any investment or trading advice or strategy,  is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance.  Only risk capital that is not required for living expenses.

__You are fully responsible for any investment or trading decisions you make__. Such decisions should be based solely on your evaluation of your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.